In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
!pip install datasets

from datasets import load_dataset

# Load the CNN/DailyMail dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

# Save dataset to Google Drive
dataset.save_to_disk("/content/drive/MyDrive/t5_dataset")

print("✅ Dataset saved to Google Drive!")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.8 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Saving the dataset (0/3 shards):   0%|          | 0/287113 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/13368 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/11490 [00:00<?, ? examples/s]

✅ Dataset saved to Google Drive!


In [5]:
from datasets import load_from_disk

# Load dataset from Google Drive
dataset = load_from_disk("/content/drive/MyDrive/t5_dataset")

print("✅ Dataset loaded from Google Drive!")


✅ Dataset loaded from Google Drive!


In [6]:
from transformers import T5Tokenizer

# Load T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Define tokenization function
def preprocess_data(examples):
    inputs = ["summarize: " + doc for doc in examples["article"]]
    targets = [summary for summary in examples["highlights"]]

    # Tokenize inputs and labels
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=150, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Save tokenized dataset to Google Drive
tokenized_dataset.save_to_disk("/content/drive/MyDrive/t5_tokenized_dataset")

print("✅ Tokenized dataset saved successfully!")


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

...

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

Saving the dataset (0/5 shards):   0%|          | 0/287113 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/13368 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/11490 [00:00<?, ? examples/s]

✅ Tokenized dataset saved successfully!


In [7]:
from datasets import load_from_disk

# Load tokenized dataset from Google Drive
tokenized_dataset = load_from_disk("/content/drive/MyDrive/t5_tokenized_dataset")

print("✅ Tokenized dataset loaded successfully!")


✅ Tokenized dataset loaded successfully!


In [8]:
!pip install deepspeed


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 31.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   

In [9]:
import json

ds_config = {
    "fp16": {
        "enabled": True
    },
    "zero_optimization": {
        "stage": 2,  # Memory optimization
        "offload_optimizer": {
            "device": "cpu"
        }
    },
    "train_batch_size": 8,
    "gradient_accumulation_steps": 2
}

# Save DeepSpeed config
with open("ds_config.json", "w") as f:
    json.dump(ds_config, f, indent=4)

print("✅ DeepSpeed config file created successfully!")


✅ DeepSpeed config file created successfully!


In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/t5_finetuned",  # Save to Google Drive
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=1000,  # Save every 1000 steps
    save_total_limit=3,
    per_device_train_batch_size=4,  # Adjust for GPU memory
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=3e-4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="/content/drive/MyDrive/t5_logs",
    logging_steps=500,
    fp16=True,  # Mixed precision
    deepspeed="ds_config.json"  # Use DeepSpeed
)

print("✅ Training arguments set successfully!")


✅ Training arguments set successfully!


In [12]:
!pip install mpi4py

from transformers import T5ForConditionalGeneration, Trainer

# Load the T5 model
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"]
)

# Start training
trainer.train()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.3/466.3 kB 35.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-4.0.3-cp311-cp311-linux_x86_64.whl size=4458242 sha256=6adae4bf4156e0e50c1fcd2dacd6adf6fc4270f4211264ba2d0dc11227551960
  Stored in directory: /root/.cache/pip/wheels/5c/56/17/bf6ba37aa971a191a8b9eaa188bf5ec855b8911c1c56fb1f84
Successfully built mpi4py


NotImplementedError: You cannot pass in a `deepspeed_plugin` when creating a second `Accelerator`. Please make sure the first `Accelerator` is initialized with all the plugins you want to use.

In [ ]:
import os
os._exit(00)  # Force a full runtime restart


In [1]:
!pip install --upgrade transformers accelerate deepspeed mpi4py


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.3.0
    Uninstalling accelerate-1.3.0:
      Successfully uninstalled accelerate-1.3.0


In [2]:
from datasets import load_from_disk
tokenized_dataset = load_from_disk("/content/drive/MyDrive/t5_tokenized_dataset")
print("✅ Tokenized dataset loaded successfully!")


✅ Tokenized dataset loaded successfully!


In [3]:
import json

ds_config = {
    "fp16": {
        "enabled": True
    },
    "zero_optimization": {
        "stage": 2,
        "offload_optimizer": {
            "device": "cpu"
        }
    },
    "train_batch_size": 8,
    "gradient_accumulation_steps": 2
}

# Save DeepSpeed config
with open("ds_config.json", "w") as f:
    json.dump(ds_config, f, indent=4)

print("✅ Updated DeepSpeed config saved!")


✅ Updated DeepSpeed config saved!


In [16]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/t5_finetuned",
    evaluation_strategy="steps",
    save_strategy="no",  # **Skip saving to speed up training**
    num_train_epochs=0.1,  # **Reduce epochs to finish faster**
    max_steps=300,  # **Limit to 300 steps (~5 minutes)**
    per_device_train_batch_size=8,  # **Reduce batch size to match DeepSpeed**
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,  # **Match DeepSpeed config**
    learning_rate=7e-4,  # **Increase learning rate for fast convergence**
    warmup_steps=20,  # **Minimal warmup for quick training**
    weight_decay=0.01,
    logging_dir="/content/drive/MyDrive/t5_logs",
    logging_steps=100,  # Log progress every 100 steps
    fp16=True,
    deepspeed="ds_config.json",
    report_to="none"
)

print("✅ Training arguments updated to match DeepSpeed settings!")


✅ Training arguments updated to match DeepSpeed settings!


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
import json

ds_config = {
    "fp16": {
        "enabled": True
    },
    "zero_optimization": {
        "stage": 1,  # Use **low optimization for speed**
        "offload_optimizer": {
            "device": "none"
        }
    },
    "train_batch_size": "auto",  # Auto-adjust based on Trainer settings
    "gradient_accumulation_steps": "auto"  # Auto-adjust to match Trainer
}

# Save the updated DeepSpeed config
with open("ds_config.json", "w") as f:
    json.dump(ds_config, f, indent=4)

print("✅ DeepSpeed config updated to auto-adjust values!")


✅ DeepSpeed config updated to auto-adjust values!


In [2]:
import json

ds_config = {
    "fp16": {
        "enabled": True
    },
    "zero_optimization": {
        "stage": 1,
        "offload_optimizer": {
            "device": "none"
        }
    },
    "train_batch_size": 16,  # Match Trainer's calculated batch size
    "gradient_accumulation_steps": 2  # Match Trainer
}

# Save the updated DeepSpeed config
with open("ds_config.json", "w") as f:
    json.dump(ds_config, f, indent=4)

print("✅ DeepSpeed config updated with explicit batch size!")


✅ DeepSpeed config updated with explicit batch size!


In [3]:
# Import necessary modules
from datasets import load_from_disk
from transformers import T5ForConditionalGeneration, TrainingArguments, Trainer

# Reload tokenized dataset
tokenized_dataset = load_from_disk("/content/drive/MyDrive/t5_tokenized_dataset")
print("✅ Tokenized dataset loaded successfully!")

# Reset DeepSpeed config
import json
ds_config = {
    "fp16": {
        "enabled": True
    },
    "zero_optimization": {
        "stage": 1,
        "offload_optimizer": {
            "device": "none"
        }
    },
    "train_batch_size": 8,
    "gradient_accumulation_steps": 2
}

# Save updated DeepSpeed config
with open("ds_config.json", "w") as f:
    json.dump(ds_config, f, indent=4)
print("✅ DeepSpeed config reset!")

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/t5_finetuned",
    evaluation_strategy="steps",
    save_strategy="no",  # Skip saving for speed
    num_train_epochs=0.1,  # Reduce epochs
    max_steps=300,  # Limit steps to ~5 mins of training
    per_device_train_batch_size=8,  # Match DeepSpeed
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,  # Match DeepSpeed
    learning_rate=7e-4,
    warmup_steps=20,
    weight_decay=0.01,
    logging_dir="/content/drive/MyDrive/t5_logs",
    logging_steps=100,
    fp16=True,
    deepspeed="ds_config.json",
    report_to="none"
)


# Load the model
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Reinitialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"]
)

print("✅ Trainer successfully reinitialized!")


✅ Tokenized dataset loaded successfully!
✅ DeepSpeed config reset!


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[2025-03-14 05:01:40,900] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)
✅ Trainer successfully reinitialized!


In [27]:
trainer.train()


ValueError: Please correct the following DeepSpeed config values that mismatch TrainingArguments values:
- ds train_batch_size=8 vs hf train_batch_size (calculated)=16
The easiest method is to set these DeepSpeed config values to 'auto'.

In [ ]:
import os
os._exit(00)  # Force full runtime restart


In [5]:
import json

ds_config = {
    "fp16": {
        "enabled": True
    },
    "zero_optimization": {
        "stage": 1,
        "offload_optimizer": {
            "device": "none"
        }
    },
    "train_micro_batch_size_per_gpu": "auto",  # Auto-adjust batch size
    "gradient_accumulation_steps": "auto"  # Auto-adjust accumulation steps
}

# Save the updated DeepSpeed config
with open("ds_config.json", "w") as f:
    json.dump(ds_config, f, indent=4)

print("✅ DeepSpeed config updated to auto-match Trainer!")



✅ DeepSpeed config updated to auto-match Trainer!


In [6]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/t5_finetuned",
    evaluation_strategy="steps",
    save_strategy="no",
    num_train_epochs=0.1,
    max_steps=300,
    per_device_train_batch_size=8,  # Trainer uses per-GPU batch size
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,  # Must match DeepSpeed settings
    learning_rate=7e-4,
    warmup_steps=20,
    weight_decay=0.01,
    logging_dir="/content/drive/MyDrive/t5_logs",
    logging_steps=100,
    fp16=True,
    deepspeed="ds_config.json",
    report_to="none"
)

print("✅ Training arguments updated!")


✅ Training arguments updated!


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [7]:
trainer.train()


AttributeError: `AcceleratorState` object has no attribute `distributed_type`. This happens if `AcceleratorState._reset_state()` was called and an `Accelerator` or `PartialState` was not reinitialized.

In [1]:
!pip install --upgrade transformers accelerate deepspeed datasets


In [2]:
import json

ds_config = {
    "fp16": {
        "enabled": True
    },
    "zero_optimization": {
        "stage": 1,
        "offload_optimizer": {
            "device": "none"
        }
    },
    "train_micro_batch_size_per_gpu": "auto",  # Let DeepSpeed auto-configure batch size
    "gradient_accumulation_steps": "auto"  # Let DeepSpeed auto-configure accumulation steps
}

# Save updated DeepSpeed config
with open("ds_config.json", "w") as f:
    json.dump(ds_config, f, indent=4)

print("✅ DeepSpeed config reset!")


✅ DeepSpeed config reset!


In [3]:
from datasets import load_from_disk
from transformers import T5ForConditionalGeneration, TrainingArguments, Trainer

# Reload tokenized dataset
tokenized_dataset = load_from_disk("/content/drive/MyDrive/t5_tokenized_dataset")
print("✅ Tokenized dataset loaded successfully!")

# Define new training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/t5_finetuned",
    evaluation_strategy="steps",
    save_strategy="no",
    num_train_epochs=0.1,
    max_steps=300,
    per_device_train_batch_size=8,  # Trainer batch size
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,  # Must match DeepSpeed settings
    learning_rate=7e-4,
    warmup_steps=20,
    weight_decay=0.01,
    logging_dir="/content/drive/MyDrive/t5_logs",
    logging_steps=100,
    fp16=True,
    deepspeed="ds_config.json",
    report_to="none"
)

# Load model
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Reinitialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"]
)

print("✅ Trainer successfully reinitialized!")


✅ Tokenized dataset loaded successfully!


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[2025-03-14 05:05:25,415] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)
✅ Trainer successfully reinitialized!


In [4]:
trainer.train()


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
100,3.938900,1.121094
200,1.107100,1.067383
300,1.049400,1.052734


TrainOutput(global_step=300, training_loss=2.0318017578125, metrics={'train_runtime': 414.801, 'train_samples_per_second': 11.572, 'train_steps_per_second': 0.723, 'total_flos': 649640647065600.0, 'train_loss': 2.0318017578125, 'epoch': 0.016717748676511563})

In [5]:
model.save_pretrained("/content/drive/MyDrive/t5_finetuned_model")
print("✅ Model saved to Google Drive successfully!")


✅ Model saved to Google Drive successfully!


In [6]:
from transformers import T5ForConditionalGeneration

# Load fine-tuned model from Google Drive
model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/t5_finetuned_model")

print("✅ Fine-tuned model loaded successfully!")


✅ Fine-tuned model loaded successfully!


In [7]:
from transformers import T5Tokenizer

# Load tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Example text input
text = "summarize: The new AI model by OpenAI has achieved breakthrough performance in NLP tasks."

# Tokenize input
inputs = tokenizer(text, return_tensors="pt")

# Generate summary
summary_ids = model.generate(inputs["input_ids"], max_length=50, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("📝 Summary:", summary)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


📝 Summary: The new AI model by OpenAI has achieved breakthrough performance in NLP tasks.


In [9]:
!pip install rouge-score sacrebleu


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.9 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=5090427161a92955e39a1dc3a5225d1734b8ed21a1ae1248fa450fbf21a667e4
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [11]:
!pip install evaluate sacrebleu rouge-score


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.1 MB/s eta 0:00:00


In [12]:
import evaluate
import numpy as np

# Load evaluation metrics
rouge = evaluate.load("rouge")
bleu = evaluate.load("sacrebleu")

# Define function to evaluate model
def evaluate_model(model, tokenizer, dataset, num_samples=50):
    references, predictions = [], []

    for i in range(num_samples):
        input_text = "summarize: " + dataset["test"][i]["article"]
        reference_summary = dataset["test"][i]["highlights"]

        inputs = tokenizer(input_text, return_tensors="pt", truncation=True)
        summary_ids = model.generate(inputs["input_ids"], max_length=50, num_beams=4, early_stopping=True)
        generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        references.append(reference_summary)
        predictions.append(generated_summary)

    # Compute ROUGE score
    rouge_scores = rouge.compute(predictions=predictions, references=references)
    print("📊 ROUGE Score:", rouge_scores)

    # Compute BLEU score
    bleu_score = bleu.compute(predictions=predictions, references=[[r] for r in references])
    print("📊 BLEU Score:", bleu_score["score"])

# Run evaluation
evaluate_model(model, tokenizer, tokenized_dataset)


📊 ROUGE Score: {'rouge1': 0.28369833238111725, 'rouge2': 0.10296882779956365, 'rougeL': 0.21523481476014725, 'rougeLsum': 0.24429231890578218}
📊 BLEU Score: 8.28613166596397


In [13]:
quantized_model = model.half()
print("✅ Model quantized to FP16 for faster inference!")


✅ Model quantized to FP16 for faster inference!


In [14]:
!pip install onnxruntime transformers[onnx]
from transformers.onnx import export
from pathlib import Path

export(
    model=model,
    tokenizer=tokenizer,
    output=Path("t5_finetuned.onnx"),
    opset=11
)
print("✅ Model converted to ONNX format!")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour 

TypeError: export() missing 2 required positional arguments: 'preprocessor' and 'config'

In [ ]:
import os
os._exit(00)  # Force a full runtime restart


In [2]:
!pip install optimum[onnxruntime]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 25.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.49.0
    Uninstalling transformers-4.49.0:
      Successfully uninstalled transformers-4.49.0


In [1]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from optimum.onnxruntime import ORTModelForSeq2SeqLM
from pathlib import Path

# Load fine-tuned model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/t5_finetuned_model")
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Define ONNX export path
onnx_path = Path("t5_finetuned.onnx")

# Convert model to ONNX format using Optimum
optimum_model = ORTModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/t5_finetuned_model")
optimum_model.export(
    onnx_path,
    tokenizer=tokenizer,
    opset=12  # Adjust opset version if needed
)

print("✅ Model successfully converted to ONNX format!")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


FileNotFoundError: Could not find any ONNX model file for the regex ['(.*)?decoder((?!(with_past|merged)).)*?\\.onnx'] in /content/drive/MyDrive/t5_finetuned_model.

In [3]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from optimum.exporters.onnx import export
from pathlib import Path

# Load fine-tuned model and tokenizer
model_path = "/content/drive/MyDrive/t5_finetuned_model"
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)

# Load model config
config = model.config  # Ensure model configuration is loaded

# Define ONNX export path
onnx_path = Path("/content/drive/MyDrive/t5_finetuned.onnx")

# Convert model to ONNX format (Now with config)
export(
    model=model,
    config=config,  # Required model configuration
    output=onnx_path,
    opset=12  # Adjust opset version if needed
)

print("✅ Model successfully converted to ONNX format!")


OSError: Can't load tokenizer for '/content/drive/MyDrive/t5_finetuned_model'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure '/content/drive/MyDrive/t5_finetuned_model' is the correct path to a directory containing all relevant files for a T5Tokenizer tokenizer.

In [2]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Define paths
model_path = "/content/drive/MyDrive/t5_finetuned_model"

# Load fine-tuned model
model = T5ForConditionalGeneration.from_pretrained(model_path)

# Save model and tokenizer
model.save_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained("t5-small")  # Reload tokenizer from base model
tokenizer.save_pretrained(model_path)

print("✅ Model and tokenizer saved successfully!")


[2025-03-14 05:37:11,231] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


✅ Model and tokenizer saved successfully!


In [3]:
model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/t5_finetuned_model")
tokenizer = T5Tokenizer.from_pretrained("/content/drive/MyDrive/t5_finetuned_model")

print("✅ Model and tokenizer loaded successfully!")


✅ Model and tokenizer loaded successfully!


In [6]:
from optimum.exporters.onnx import export
from pathlib import Path

# Define ONNX export path
onnx_path = Path("/content/drive/MyDrive/t5_finetuned.onnx")

# Convert model to ONNX format
export(
    model=model,
    config=model.config,  # Ensure model configuration is loaded
    output=onnx_path,
    opset=12
)

print("✅ Model successfully converted to ONNX format!")


AttributeError: 'T5Config' object has no attribute 'is_transformers_support_available'

In [5]:

!pip install --upgrade optimum transformers onnxruntime onnx


In [ ]:
import os
os._exit(00)  # Restart the runtime to apply updates


In [6]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from optimum.onnxruntime import ORTModelForSeq2SeqLM
from pathlib import Path

# Define paths
model_path = "/content/drive/MyDrive/t5_finetuned_model"
onnx_path = Path("/content/drive/MyDrive/t5_finetuned")

# Load fine-tuned model and tokenizer
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)

# Convert model to ONNX format using Optimum
optimum_model = ORTModelForSeq2SeqLM.from_pretrained(model_path, from_transformers=True)
optimum_model.save_pretrained(onnx_path)

print("✅ Model successfully converted to ONNX format!")


The argument `from_transformers` is deprecated, and will be removed in optimum 2.0.  Use `export` instead
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.11/dist-packages/transformers/models/t5/modeling_t5.py:1306: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if sequence_length != 1:
/usr/local/lib/python3.11/dist-packages/transformers/cache_utils.py:451: TracerWarning: Using len to get tensor shape might cause the trace to be incorrect. Recommended usage would be tensor.s

✅ Model successfully converted to ONNX format!


In [8]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from optimum.onnxruntime import ORTModelForSeq2SeqLM
from pathlib import Path

# Define paths
model_path = "/content/drive/MyDrive/t5_finetuned_model"
onnx_path = Path("/content/drive/MyDrive/t5_finetuned_onnx")

# Load fine-tuned model and tokenizer
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)

# Convert model to ONNX format using Optimum
optimum_model = ORTModelForSeq2SeqLM.from_pretrained(model_path, from_transformers=True)
optimum_model.save_pretrained(onnx_path)

print("✅ Model successfully converted to ONNX format and saved at:", onnx_path)


The argument `from_transformers` is deprecated, and will be removed in optimum 2.0.  Use `export` instead
/usr/local/lib/python3.11/dist-packages/transformers/models/t5/modeling_t5.py:1306: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if sequence_length != 1:
/usr/local/lib/python3.11/dist-packages/transformers/cache_utils.py:451: TracerWarning: Using len to get tensor shape might cause the trace to be incorrect. Recommended usage would be tensor.shape[0]. Passing a tensor of different shape might lead to errors or silently give incorrect results.
  or len(self.key_cache[layer_idx]) == 0  # the layer has no cache
/usr/local/lib/python3.11/dist-packages/transformers/cache_utils.py:435: TracerWarning: Using len to get tensor shape might cause the trace to be incorrect. 

✅ Model successfully converted to ONNX format and saved at: /content/drive/MyDrive/t5_finetuned_onnx


In [9]:
!ls /content/drive/MyDrive/t5_finetuned_onnx


config.json		      encoder_model.onnx       spiece.model
decoder_model.onnx	      generation_config.json   tokenizer_config.json
decoder_with_past_model.onnx  special_tokens_map.json  tokenizer.json


In [7]:
import onnx

# Load the ONNX model
onnx_model = onnx.load("/content/drive/MyDrive/t5_finetuned/model.onnx")

# Check model structure
onnx.checker.check_model(onnx_model)

print("✅ ONNX model is valid and ready for deployment!")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/t5_finetuned/model.onnx'

In [10]:
import onnxruntime as ort
from transformers import T5Tokenizer

# Define model paths
onnx_model_dir = "/content/drive/MyDrive/t5_finetuned_onnx"
encoder_path = f"{onnx_model_dir}/encoder_model.onnx"
decoder_path = f"{onnx_model_dir}/decoder_model.onnx"
decoder_with_past_path = f"{onnx_model_dir}/decoder_with_past_model.onnx"

# Load ONNX sessions
encoder_session = ort.InferenceSession(encoder_path)
decoder_session = ort.InferenceSession(decoder_path)
decoder_with_past_session = ort.InferenceSession(decoder_with_past_path)

# Load tokenizer
tokenizer = T5Tokenizer.from_pretrained(onnx_model_dir)

print("✅ ONNX model loaded successfully!")


✅ ONNX model loaded successfully!


In [12]:
import onnxruntime as ort
from transformers import T5Tokenizer
import numpy as np

# Define model paths
onnx_model_dir = "/content/drive/MyDrive/t5_finetuned_onnx"
encoder_path = f"{onnx_model_dir}/encoder_model.onnx"
decoder_path = f"{onnx_model_dir}/decoder_model.onnx"

# Load ONNX sessions
encoder_session = ort.InferenceSession(encoder_path)
decoder_session = ort.InferenceSession(decoder_path)

# Load tokenizer
tokenizer = T5Tokenizer.from_pretrained(onnx_model_dir)

def onnx_summarize(text):
    inputs = tokenizer("summarize: " + text, return_tensors="np")

    # Encode input (fix: include attention_mask)
    ort_inputs = {
        encoder_session.get_inputs()[0].name: inputs["input_ids"],
        encoder_session.get_inputs()[1].name: inputs["attention_mask"]  # Fix: include attention mask
    }
    encoder_outputs = encoder_session.run(None, ort_inputs)

    # Decode output
    decoder_inputs = {decoder_session.get_inputs()[0].name: encoder_outputs[0]}
    decoder_outputs = decoder_session.run(None, decoder_inputs)

    # Generate summary
    summary = tokenizer.decode(decoder_outputs[0][0], skip_special_tokens=True)
    return summary

# Test ONNX model
test_text = "The new AI model by OpenAI has achieved breakthrough performance in NLP tasks."
summary = onnx_summarize(test_text)

print("📝 ONNX Model Summary:", summary)


ValueError: Required inputs (['input_ids', 'encoder_hidden_states']) are missing from input feed (['encoder_attention_mask']).

In [14]:
import onnxruntime as ort
from transformers import T5Tokenizer
import numpy as np

# Define model paths
onnx_model_dir = "/content/drive/MyDrive/t5_finetuned_onnx"
encoder_path = f"{onnx_model_dir}/encoder_model.onnx"
decoder_path = f"{onnx_model_dir}/decoder_model.onnx"

# Load ONNX sessions
encoder_session = ort.InferenceSession(encoder_path)
decoder_session = ort.InferenceSession(decoder_path)

# Load tokenizer
tokenizer = T5Tokenizer.from_pretrained(onnx_model_dir)

def onnx_summarize(text):
    inputs = tokenizer("summarize: " + text, return_tensors="np")

    # Encode input (Ensure both input_ids & attention_mask are included)
    encoder_inputs = {
        encoder_session.get_inputs()[0].name: inputs["input_ids"].astype(np.int64),
        encoder_session.get_inputs()[1].name: inputs["attention_mask"].astype(np.int64)
    }
    encoder_outputs = encoder_session.run(None, encoder_inputs)

    # Decoder needs encoder's hidden states and a starting token (<pad>)
    decoder_inputs = {
        decoder_session.get_inputs()[0].name: np.array([[0]], dtype=np.int64),  # <pad> token as decoder start token
        decoder_session.get_inputs()[1].name: encoder_outputs[0]  # Encoder hidden states
    }
    decoder_outputs = decoder_session.run(None, decoder_inputs)

    # Generate summary
    summary = tokenizer.decode(decoder_outputs[0][0], skip_special_tokens=True)
    return summary

# Test ONNX model
test_text = "The new AI model by OpenAI has achieved breakthrough performance in NLP tasks."
summary = onnx_summarize(test_text)

print("📝 ONNX Model Summary:", summary)


ValueError: Required inputs (['encoder_hidden_states']) are missing from input feed (['encoder_attention_mask', 'input_ids']).

In [15]:
# Check encoder inputs
print("Encoder model inputs:")
for input_tensor in encoder_session.get_inputs():
    print(f"- {input_tensor.name}: {input_tensor.shape}")

# Check decoder inputs
print("\nDecoder model inputs:")
for input_tensor in decoder_session.get_inputs():
    print(f"- {input_tensor.name}: {input_tensor.shape}")


Encoder model inputs:
- input_ids: ['batch_size', 'encoder_sequence_length']
- attention_mask: ['batch_size', 'encoder_sequence_length']

Decoder model inputs:
- encoder_attention_mask: ['batch_size', 'encoder_sequence_length']
- input_ids: ['batch_size', 'decoder_sequence_length']
- encoder_hidden_states: ['batch_size', 'encoder_sequence_length', 512]


In [17]:
import onnxruntime as ort
from transformers import T5Tokenizer
import numpy as np

# Define model paths
onnx_model_dir = "/content/drive/MyDrive/t5_finetuned_onnx"
encoder_path = f"{onnx_model_dir}/encoder_model.onnx"
decoder_path = f"{onnx_model_dir}/decoder_model.onnx"

# Load ONNX sessions
encoder_session = ort.InferenceSession(encoder_path)
decoder_session = ort.InferenceSession(decoder_path)

# Load tokenizer
tokenizer = T5Tokenizer.from_pretrained(onnx_model_dir)

def onnx_summarize(text):
    inputs = tokenizer("summarize: " + text, return_tensors="np")

    # Convert inputs to numpy int64 (required by ONNX)
    input_ids = inputs["input_ids"].astype(np.int64)
    attention_mask = inputs["attention_mask"].astype(np.int64)

    # Encode input
    encoder_inputs = {
        "input_ids": input_ids,
        "attention_mask": attention_mask
    }
    encoder_outputs = encoder_session.run(None, encoder_inputs)

    # Prepare decoder inputs
    decoder_input_ids = np.array([[0]], dtype=np.int64)  # <pad> token as start

    # Pass encoder outputs to decoder
    decoder_inputs = {
        "input_ids": decoder_input_ids,  # Decoder input tokens
        "encoder_hidden_states": encoder_outputs[0],  # Pass encoder output
        "encoder_attention_mask": attention_mask  # Use same attention mask
    }

    decoder_outputs = decoder_session.run(None, decoder_inputs)

    # Fix: Ensure the output is a NumPy array before converting to tokens
    decoded_ids = np.array(decoder_outputs[0][0])  # Convert list to NumPy array

    # Generate summary
    summary = tokenizer.decode(decoded_ids, skip_special_tokens=True)
    return summary

# Test ONNX model
test_text = "The new AI model by OpenAI has achieved breakthrough performance in NLP tasks."
summary = onnx_summarize(test_text)

print("📝 ONNX Model Summary:", summary)


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'list'

In [22]:
import onnxruntime as ort
from transformers import T5Tokenizer
import numpy as np

# Define model paths
onnx_model_dir = "/content/drive/MyDrive/t5_finetuned_onnx"
encoder_path = f"{onnx_model_dir}/encoder_model.onnx"
decoder_path = f"{onnx_model_dir}/decoder_model.onnx"

# Load ONNX sessions
encoder_session = ort.InferenceSession(encoder_path)
decoder_session = ort.InferenceSession(decoder_path)

# Load tokenizer
tokenizer = T5Tokenizer.from_pretrained(onnx_model_dir)

def onnx_summarize(text, max_length=50):
    inputs = tokenizer("summarize: " + text, return_tensors="np")

    # Convert inputs to numpy int64 (required by ONNX)
    input_ids = inputs["input_ids"].astype(np.int64)
    attention_mask = inputs["attention_mask"].astype(np.int64)

    # Encode input
    encoder_inputs = {
        "input_ids": input_ids,
        "attention_mask": attention_mask
    }
    encoder_outputs = encoder_session.run(None, encoder_inputs)

    # Initialize decoder input with <pad> token
    decoder_input_ids = np.array([[tokenizer.pad_token_id]], dtype=np.int64)

    generated_tokens = []

    for _ in range(max_length):
        # Prepare decoder inputs
        decoder_inputs = {
            "input_ids": decoder_input_ids,
            "encoder_hidden_states": encoder_outputs[0],
            "encoder_attention_mask": attention_mask
        }

        decoder_outputs = decoder_session.run(None, decoder_inputs)

        # Fix: Extract only the LAST generated token from the sequence
        next_token_id = int(decoder_outputs[0][0][-1])  # Ensure only last token is used

        # Stop if EOS token is generated
        if next_token_id == tokenizer.eos_token_id:
            break

        # Append token to generated sequence
        generated_tokens.append(next_token_id)

        # Update decoder input with the new token
        decoder_input_ids = np.array([generated_tokens], dtype=np.int64)

    # Convert generated tokens into text
    summary = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return summary

# Test ONNX model
test_text = "The new AI model by OpenAI has achieved breakthrough performance in NLP tasks."
summary = onnx_summarize(test_text)

print("📝 ONNX Model Summary:", summary)


TypeError: only length-1 arrays can be converted to Python scalars

In [24]:
def onnx_summarize(text, max_length=50):
    inputs = tokenizer("summarize: " + text, return_tensors="np")

    # Convert inputs to numpy int64 (required by ONNX)
    input_ids = inputs["input_ids"].astype(np.int64)
    attention_mask = inputs["attention_mask"].astype(np.int64)

    # Encode input
    encoder_inputs = {
        "input_ids": input_ids,
        "attention_mask": attention_mask
    }
    encoder_outputs = encoder_session.run(None, encoder_inputs)

    # Initialize decoder input with <pad> token
    decoder_input_ids = np.array([[tokenizer.pad_token_id]], dtype=np.int64)

    generated_tokens = []

    for _ in range(max_length):
        # Prepare decoder inputs
        decoder_inputs = {
            "input_ids": decoder_input_ids,
            "encoder_hidden_states": encoder_outputs[0],
            "encoder_attention_mask": attention_mask
        }

        decoder_outputs = decoder_session.run(None, decoder_inputs)

        # Debug: Print decoder output structure
        print(f"Decoder Output Shape: {np.array(decoder_outputs).shape}")
        print(f"Decoder Output Example: {decoder_outputs}")

        # Extract last generated token correctly
        output_array = np.array(decoder_outputs[0])  # Ensure output is a NumPy array
        next_token_id = int(output_array[0, -1])  # Extract last token from sequence

        # Stop if EOS token is generated
        if next_token_id == tokenizer.eos_token_id:
            break

        # Append token to generated sequence
        generated_tokens.append(next_token_id)

        # Update decoder input with the new token
        decoder_input_ids = np.expand_dims(np.array(generated_tokens, dtype=np.int64), axis=0)

    # Convert generated tokens into text
    summary = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return summary


In [25]:
test_text = "The new AI model by OpenAI has achieved breakthrough performance in NLP tasks."
summary = onnx_summarize(test_text)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (25, 1) + inhomogeneous part.

In [30]:
def onnx_summarize(text, max_length=50):
    inputs = tokenizer("summarize: " + text, return_tensors="np")

    # Convert inputs to numpy int64 (required by ONNX)
    input_ids = inputs["input_ids"].astype(np.int64)
    attention_mask = inputs["attention_mask"].astype(np.int64)

    # Encode input
    encoder_inputs = {
        "input_ids": input_ids,
        "attention_mask": attention_mask
    }
    encoder_outputs = encoder_session.run(None, encoder_inputs)

    # Initialize decoder input with <pad> token
    decoder_input_ids = np.array([[tokenizer.pad_token_id]], dtype=np.int64)

    generated_tokens = []

    for _ in range(max_length):
        # Prepare decoder inputs
        decoder_inputs = {
            "input_ids": decoder_input_ids,
            "encoder_hidden_states": encoder_outputs[0],
            "encoder_attention_mask": attention_mask
        }

        decoder_outputs = decoder_session.run(None, decoder_inputs)

        # Debug: Print actual ONNX decoder output
        print(f"Decoder Output Type: {type(decoder_outputs)}")
        print(f"Decoder Output Shape: {np.array(decoder_outputs, dtype=object).shape}")
        print(f"Decoder Output Example: {decoder_outputs}")

        break  # Stop early to inspect output

# Test ONNX model
test_text = "The new AI model by OpenAI has achieved breakthrough performance in NLP tasks."
onnx_summarize(test_text)


Decoder Output Type: <class 'list'>


ValueError: could not broadcast input array from shape (32128,) into shape (1,)

In [31]:
!pip install fastapi uvicorn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 8.5 MB/s eta 0:00:00


In [33]:
!pip install fastapi uvicorn nest_asyncio pyngrok transformers torch


In [4]:
!ngrok authtoken 2uINrrFcB9oWoA8HHa5LJDgYvr8_3skwYnJ9xiabGsjdvJWAZ


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import os
os._exit(00)  # Force restart


In [3]:
!pkill -f uvicorn
!pkill -f ngrok


In [5]:
from fastapi import FastAPI
from transformers import T5Tokenizer, T5ForConditionalGeneration
import uvicorn
import nest_asyncio
import threading
from pyngrok import ngrok

# Initialize FastAPI app
app = FastAPI()

# Load fine-tuned model
model_path = "/content/drive/MyDrive/t5_finetuned_model"
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)

@app.get("/summarize/")
def summarize(text: str):
    inputs = tokenizer("summarize: " + text, return_tensors="pt")
    summary_ids = model.generate(inputs["input_ids"], max_length=50, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return {"summary": summary}

# Fix: Run Uvicorn inside a thread to avoid Colab async conflicts
def run_api():
    nest_asyncio.apply()
    uvicorn.run(app, host="0.0.0.0", port=5000)  # Use port 5000 to avoid conflicts

# Start FastAPI in a background thread
thread = threading.Thread(target=run_api, daemon=True)
thread.start()

# Expose the API using Ngrok (Now using port 5000)
public_url = ngrok.connect(5000).public_url
print(f"🚀 Public API URL: {public_url}")


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-4' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=SystemExit(1)>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 163, in startup
    server = await loop.create_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 1536, in create_server
    raise OSError(err.errno, msg) from None
OSError: [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-acbff26307bb>", line 26, in run_a

🚀 Public API URL: https://b87e-34-143-158-150.ngrok-free.app
